# Generating C++ Kokkos Chemistry Solvers with JAFF

This notebook demonstrates how to use JAFF to generate C++ code that uses the Kokkos-kernels BDF solver for solving chemical reaction networks. We'll use the `react_COthin` network as an example, which contains 37 species and 287 reactions for carbon-oxygen chemistry.

## Overview

The Kokkos output plugin generates:
- **chemistry_ode.hpp**: Chemistry ODE system class with evaluate_function and evaluate_jacobian methods
- **chemistry_ode.cpp**: Main executable that sets up and runs the BDF solver
- **CMakeLists.txt**: Build configuration for Kokkos and Kokkos-kernels dependencies

The generated code is compatible with the Kokkos-kernels BDF solver for stiff ODEs.

## Step 1: Load the Chemical Network

First, let's load the react_COthin network, which is a carbon-oxygen chemistry network in KROME format.

In [1]:
from jaff.network import Network
from jaff.builder import Builder
import os

# Load the react_COthin network
network = Network('../networks/react_COthin')

print(f"Network loaded: {network.label}")
print(f"Number of species: {network.get_number_of_species()}")
print(f"Number of reactions: {len(network.reactions)}")

Welcome to JAFF: Just Another Fancy Format!
Loading network from ../networks/react_COthin
Network label = react_COthin


  8%|█████████▌                                                                                                                  | 27/350 [00:00<00:01, 187.10it/s]

KROME variable detected: @var:Hnuclei = get_Hnuclei(n(:))
KROME variable detected: @var: Te = Tgas*8.617343d-5
KROME variable detected: @var: invT = 1d0/Tgas
KROME variable detected: @var: lnTe = log(Te)
KROME variable detected: @var: T = Tgas
KROME format detected: @format:idx,R,R,R,P,P,P,P,Tmin,Tmax,rate
exp(-2.03914985e-6*lnTe**8 + 0.000111954395*lnTe**7 - 0.00263197617*lnTe**6 + 0.0348255977*lnTe**5 - 0.2877056*lnTe**4 + 1.56315498*lnTe**3 - 5.73932875*lnTe**2 + 13.536556*lnTe - 32.71396786)
3.92e-13*invTe**0.6353
exp(-3.071135243196595e-9*lnTe**9 - 1.856767039775261e-8*lnTe**8 + 5.755614137575758e-7*lnTe**7 + 4.989108920299513e-6*lnTe**6 - 1.421502914054107e-5*lnTe**5 - 0.0003212605213188796*lnTe**4 - 0.002380861877349834*lnTe**3 - 0.02026044731984691*lnTe**2 - 0.7241125657826851*lnTe - 28.61303380689232)
exp(-3.64916141e-6*lnTe**8 + 0.000206723616*lnTe**7 - 0.0050090561*lnTe**6 + 0.0679539123*lnTe**5 - 0.56851189*lnTe**4 + 3.05803875*lnTe**3 - 10.7532302*lnTe**2 + 23.91596563*lnT

 24%|█████████████████████████████▍                                                                                              | 83/350 [00:00<00:01, 214.53it/s]

6.9e-32/Tgas**0.4
1.3e-32/T32**0.38
1.3e-32/T32**1.0
1.3e-32/(8.0*T32**0.38)
1.3e-32/(8.0*T32**1.0)
4.67e-12/T32**0.6
1.23e-17*T32**2.49*exp(21845.6*invT)
9.62e-8*exp(-115786.2*invT)/T32**1.37
7.5e-12/T32**0.55
4.86e-12/T32**0.32
1.3e-10/Tgas**0.64
0.00074*(1.0 + 0.062*exp(-145000.0*invT))*exp(-175000.0*invT)/Tgas**1.5 + 1.41e-10/Tgas**0.66
KROME variable detected: @var:u1 = 11.26d0*invTe
KROME variable detected: @var:u2 = 8.2d0*invTe
KROME variable detected: @var:u3 = 13.6*invTe
KROME format detected: @format:idx,R,R,R,P,P,P,P,Tmin,Tmax,rate
6.85e-8*u1**0.25*exp(-u1)/(u1 + 0.193)
1.88e-7*u2**0.25*(u2**0.5 + 1.0)*exp(-u2)/(u2 + 0.376)
3.59e-8*u3**0.34*exp(-u3)/(u3 + 0.073)
4.99e-11*Tgas**0.405 + 7.54e-10*invT**0.458
(4.0e-10*Tgas**0.00669 + 1.08e-11*Tgas**0.517)*exp(-227.0*invT)
2.78e-15*exp(-Tgas*1.2258e-6)/(Tgas*0.0001)**0.2163 + 4.991e-15*(Tgas*0.0001)**0.3794*exp(-Tgas*8.9206e-7)
3.9e-16*Tgas**0.213
6.08e-14*(Tgas*0.0001)**1.96*exp(-170000.0*invT)
8.58e-17*Tgas**0.757
3.25e-17*Tgas

...

2.4e-11*exp(-110.0*invT)
2.42000000000000e-12
6.2e-14*T32**2.62*exp(-945.0*invT)
1.38000000000000e-12
1.65e-12*T32**1.14*exp(-50.0*invT)
1.59e-11*T32*1.2*exp(-9610.0*invT)
2.61e-10*1.2*exp(-8156.0*invT)
3.16e-10*exp(-21890.0*invT)
4.7e-11/T32**0.34
2.48e-12*T32**1.54*exp(613.0*invT)
1.1e-10*T32**0.5*exp(-77700.0*invT)
2.24e-9*T32**0.042*exp((-Tgas)/46600.0)
7.7e-9*exp(-17560.0*invT)
2.40000000000000e-9
(7.2733e-11*sqrt(Tgas) + 5.9203e-14*Tgas + 1.0218e-9)/(Tgas**0.1667 + 0.044914*sqrt(Tgas) - 5.9203e-14*Tgas + 2.6397e-6*Tgas**1.5)
8.5145e-10/(Tgas**0.1667 + 0.00095666*sqrt(Tgas) - 4.404e-5*Tgas + 2.3496e-6*Tgas**1.5)
1.0e-10*exp(-4640.0*invT)
7.50000000000000e-10
1.20000000000000e-9
3.50000000000000e-10
1.0e-9*exp(-7080.0*invT)
1.60000000000000e-9
7.50000000000000e-10
7.0e-10*exp(-10560.0*invT)
2.50000000000000e-10
2.50000000000000e-10
4.80000000000000e-10
1.69000000000000e-9
1.50000000000000e-9
7.98e-10*exp(-1.41*invT)/T32**0.156
3.42e-10*exp(-1.41*invT)/T32**0.156
2.28000000000000e-8

 67%|██████████████████████████████████████████████████████████████████████████████████▏                                        | 234/350 [00:00<00:00, 531.74it/s]

9.65e-10*(0.672147*(300.0*invT)**0.5 + 0.62)
9.65e-10*(56.6625498765*invT + 0.136347*(300.0*invT)**0.5 + 1.0)
1.60000000000000e-9
KROME format detected: @format:idx,R,R,P,P,Tmin,Tmax,rate
3.75000000000000e-8
2.5e-9*(2.62185*(300.0*invT)**0.5 + 0.62)
KROME format detected: @format:idx,R,R,P,P,P,Tmin,Tmax,rate
2.20000000000000e-8
1.35e-9*(2.62185*(300.0*invT)**0.5 + 0.62)
KROME format detected: @format:idx,R,R,P,P,Tmin,Tmax,rate
4.20000000000000e-8
2.85e-9*(2.578947*(300.0*invT)**0.5 + 0.62)
KROME format detected: @format:idx,R,R,P,P,P,Tmin,Tmax,rate
7.56000000000000e-9
5.1282e-10*(2.578947*(300.0*invT)**0.5 + 0.62)
7.56000000000000e-9
5.1282e-10*(2.578947*(300.0*invT)**0.5 + 0.62)
KROME format detected: @format:idx,R,R,P,P,Tmin,Tmax,rate
7.56000000000000e-9
5.1282e-10*(2.578947*(300.0*invT)**0.5 + 0.62)
2.00000000000000e-9
3.30000000000000e-11
KROME format detected: @format:idx,R,R,P,P,P,Tmin,Tmax,rate
1.10000000000000e-9
1.4e-9/T32**0.5
1.4e-16/T32**0.5
KROME format detected: @format:i

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 350/350 [00:00<00:00, 469.76it/s]

237.0*crate
0.1*crate
0.0003*crate
0.93*crate
1020.0*crate
730.0*crate
750.0*crate
117.0*crate
510.0*crate
500.0*crate
970.0*crate
421.0*crate
1170.0*crate
KROME format detected: @format:idx,R,P,P,P,rate
0.93*crate
KROME variable detected: @var:ntot=sum(n(1:nmols))
KROME format detected: @format:idx,R,R,P,Tmin,Tmax,rate
5.99e-33*ntot/(Tgas/5000.0)**1.6
5.99e-33*ntot*exp(5255.0/Tgas)/(Tgas/5000.0)**0.64
6.16e-29*ntot/(Tgas/300.0)**3.08
2.14e-29*ntot*exp(2114.0/Tgas)/(Tgas/300.0)**3.08
6.16e-27*ntot/(Tgas/300.0)**3.08
2.14e-27*ntot*exp(2114.0/Tgas)/(Tgas/300.0)**3.08
6.16e-27*ntot/(Tgas/300.0)**3.08
2.14e-27*ntot*exp(2114.0/Tgas)/(Tgas/300.0)**3.08
4.33e-32*ntot/T32
2.56e-31*ntot/T32**2
9.2e-34*ntot/T32
Variables found: ['HnOj', 'Hnuclei', 'T', 'T32', 'Te', 'Tgas', 'crate', 'fA', 'idx_H', 'invT', 'invT32', 'invTe', 'invTgas', 'lnTe', 'n', 'ntot', 'sqrTgas', 'tgas', 'user_Av', 'user_H2self', 'user_Tdust']
Loaded 287 reactions
Lodaded 0 photo-chemistry reactions
All done!
Network loaded: r

## Step 2: Examine Network Properties

Let's examine some properties of the network to understand what we're working with.

In [2]:
# Show the first few species
print("First 10 species:")
for i, species in enumerate(network.species[:10]):
    print(f"  {i}: {species.name} (mass: {species.mass:.2f}, charge: {species.charge})")

if len(network.species) > 10:
    print(f"  ... and {len(network.species) - 10} more species")

print(f"\nTotal species: {len(network.species)}")

First 10 species:
  0: H (mass: 0.00, charge: 0)
  1: e- (mass: 0.00, charge: -1)
  2: H+ (mass: 0.00, charge: 1)
  3: He (mass: 0.00, charge: 0)
  4: He+ (mass: 0.00, charge: 1)
  5: He++ (mass: 0.00, charge: 2)
  6: H2 (mass: 0.00, charge: 0)
  7: H2+ (mass: 0.00, charge: 1)
  8: H- (mass: 0.00, charge: -1)
  9: C+ (mass: 0.00, charge: 1)
  ... and 27 more species

Total species: 37


In [3]:
# Show a few example reactions
print("First 5 reactions:")
for i, reaction in enumerate(network.reactions[:5]):
    reactants = " + ".join([f"{r.name}" for r in reaction.reactants])
    products = " + ".join([f"{p.name}" for p in reaction.products])
    print(f"  {i+1}: {reactants} -> {products}")
    print(f"      Rate: {reaction.rate}")
    print()

First 5 reactions:
  1: H + e- -> H+ + e- + e-
      Rate: exp(-2.03914985e-6*lnTe**8 + 0.000111954395*lnTe**7 - 0.00263197617*lnTe**6 + 0.0348255977*lnTe**5 - 0.2877056*lnTe**4 + 1.56315498*lnTe**3 - 5.73932875*lnTe**2 + 13.536556*lnTe - 32.71396786)

  2: H+ + e- -> H
      Rate: 3.92e-13*invTe**0.6353

  3: H+ + e- -> H
      Rate: exp(-3.071135243196595e-9*lnTe**9 - 1.856767039775261e-8*lnTe**8 + 5.755614137575758e-7*lnTe**7 + 4.989108920299513e-6*lnTe**6 - 1.421502914054107e-5*lnTe**5 - 0.0003212605213188796*lnTe**4 - 0.002380861877349834*lnTe**3 - 0.02026044731984691*lnTe**2 - 0.7241125657826851*lnTe - 28.61303380689232)

  4: He + e- -> He+ + e- + e-
      Rate: exp(-3.64916141e-6*lnTe**8 + 0.000206723616*lnTe**7 - 0.0050090561*lnTe**6 + 0.0679539123*lnTe**5 - 0.56851189*lnTe**4 + 3.05803875*lnTe**3 - 10.7532302*lnTe**2 + 23.91596563*lnTe - 44.09864886)

  5: He+ + e- -> He
      Rate: 3.92e-13*invTe**0.6353



## Step 3: Generate C++ Kokkos Code

Now let's use the Builder to generate C++ code using the `kokkos_ode` template.

In [4]:
# Create builder and generate Kokkos C++ code
builder = Builder(network)
builds_dir = builder.build(template="kokkos_ode")

print("\nC++ Kokkos code generated successfully!")
print(f"Generated files are located at {builds_dir}")

Building network with template: kokkos_ode
Preprocessing /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/templates/kokkos_ode/chemistry_ode.hpp -> /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/builds/chemistry_ode.hpp
Preprocessing /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/templates/kokkos_ode/chemistry_ode.cpp -> /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/builds/chemistry_ode.cpp
Preprocessing /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/templates/kokkos_ode/CMakeLists.txt -> /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/builds/CMakeLists.txt
Network built successfully using template 'kokkos_ode'.
Output files are located in: /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/builds

C++ Kokkos code generated successfully!
Generated files are located at /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/builds


## Step 4: Examine Generated Files

Let's look at the structure and content of the generated files.

In [5]:
# List generated files
generated_files = [f for f in os.listdir(builds_dir) if f.endswith(('.cpp', '.hpp', '.txt'))]

print("Generated files:")
for file in sorted(generated_files):
    filepath = os.path.join(builds_dir, file)
    size = os.path.getsize(filepath)
    print(f"  {file:20} ({size:,} bytes)")

Generated files:
  CMakeLists.txt       (1,309 bytes)
  chemistry_ode.cpp    (6,000 bytes)
  chemistry_ode.hpp    (56,994 bytes)


## Step 5: Understanding the Generated Code Structure

Let's examine key components of the generated chemistry solver.

In [6]:
# Show the ChemistryODE class definition
with open(builds_dir + "/chemistry_ode.hpp", 'r') as f:
    lines = f.readlines()

# Find the class definition
class_start = None
for i, line in enumerate(lines):
    if "struct ChemistryODE" in line:
        class_start = i
        break

if class_start:
    print("ChemistryODE class structure:")
    print("=" * 50)
    # Show class definition and methods
    brace_count = 0
    for i in range(class_start, min(class_start + 100, len(lines))):
        line = lines[i].rstrip()
        print(f"{i+1:2d}: {line}")
        
        if '{' in line:
            brace_count += line.count('{')
        if '}' in line:
            brace_count -= line.count('}')
            if brace_count == 0:
                break

ChemistryODE class structure:
13: struct ChemistryODE {
14:     // Number of species in the chemical network
15:     // PREPROCESS_NUM_SPECIES
16: 
17:     static constexpr int neqs = 37;
18: 
19:     // PREPROCESS_END
20: 
21:     // Species indices
22:     // PREPROCESS_COMMONS
23: 
24:     static constexpr int idx_H = 0;
25:     static constexpr int idx_e = 1;
26:     static constexpr int idx_Hj = 2;
27:     static constexpr int idx_He = 3;
28:     static constexpr int idx_Hej = 4;
29:     static constexpr int idx_Hejj = 5;
30:     static constexpr int idx_H2 = 6;
31:     static constexpr int idx_H2j = 7;
32:     static constexpr int idx_Hk = 8;
33:     static constexpr int idx_Cj = 9;
34:     static constexpr int idx_C = 10;
35:     static constexpr int idx_Sij = 11;
36:     static constexpr int idx_Si = 12;
37:     static constexpr int idx_Oj = 13;
38:     static constexpr int idx_O = 14;
39:     static constexpr int idx_Sijj = 15;
40:     static constexpr int idx_OH = 16;
41:    

In [7]:
# Show main function structure from chemistry_ode.cpp
with open(builds_dir + "/chemistry_ode.cpp", 'r') as f:
    lines = f.readlines()

# Find main function
main_start = None
for i, line in enumerate(lines):
    if "int main(" in line:
        main_start = i
        break

if main_start:
    print("Main function structure (first 25 lines):")
    print("=" * 50)
    for i in range(main_start, min(main_start + 25, len(lines))):
        line = lines[i].rstrip()
        print(f"{i+1:2d}: {line}")

Main function structure (first 25 lines):
41: int main(int argc, char* argv[]) {
42:     Kokkos::initialize(argc, argv);
43: 
44:     {
45:         using execution_space = Kokkos::DefaultExecutionSpace;
46:         using scalar_type = double;
47:         using vec_type = Kokkos::View<scalar_type*, execution_space>;
48:         using mat_type = Kokkos::View<scalar_type**, execution_space>;
49: 
50:         ChemistryODE mySys{};
51: 
52:         // Time integration parameters
53:         const scalar_type t_start = 0.0;
54:         scalar_type t_end = 3.15576e9; // 100 years in seconds default
55: 
56:         // Parse command line arguments
57:         if (argc > 1) {
58:             t_end = std::stod(argv[1]);
59:         }
60: 
61:         std::cout << "Solving Chemistry ODE System\n";
62:         std::cout << "Number of species: " << mySys.neqs << "\n";
63:         std::cout << "Number of reactions: " <<
64: // PREPROCESS_NUM_REACTIONS
65: 


## Step 6: Building and Running Instructions

Here's how to build and run the generated C++ code:

In [8]:
print(f"""To build and run the generated C++ chemistry solver:

1. Configure the build with CMake:
   ```
   cd {builds_dir}
   cmake -S . -B build
   ```
2. Build with parallel compilation:
   ```
   cmake --build build -j
   ```
3. Run the chemistry solver:
   ```
   ./build/chemistry_ode
   ```
   Or, run with custom time interval (100 years in seconds):
   ```
   ./build/chemistry_ode 3.15576e9
   ```

The solver uses the Kokkos-kernels BDF solver which is well-suited for
stiff chemical reaction networks. It automatically adapts time steps
and uses numerical differentiation for the Jacobian.""")

To build and run the generated C++ chemistry solver:

1. Configure the build with CMake:
   ```
   cd /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/builds
   cmake -S . -B build
   ```
2. Build with parallel compilation:
   ```
   cmake --build build -j
   ```
3. Run the chemistry solver:
   ```
   ./build/chemistry_ode
   ```
   Or, run with custom time interval (100 years in seconds):
   ```
   ./build/chemistry_ode 3.15576e9
   ```

The solver uses the Kokkos-kernels BDF solver which is well-suited for
stiff chemical reaction networks. It automatically adapts time steps
and uses numerical differentiation for the Jacobian.
